In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
path = '/home/rupeshr/Desktop/TSA_Python/dataset/time series dataset/Electric_Production1.csv'
df = pd.read_csv(path, parse_dates=['DATE'], index_col='DATE')

In [42]:
df.head()

,Unnamed: 0,IPG2211A2N
DATE,,
1985-01-01,0,72.5052
1985-02-01,1,70.6720
1985-03-01,2,62.4502
1985-04-01,3,57.4714
1985-05-01,4,55.3151


In [43]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [44]:
df.head()

,IPG2211A2N
DATE,
1985-01-01,72.5052
1985-02-01,70.6720
1985-03-01,62.4502
1985-04-01,57.4714
1985-05-01,55.3151


In [45]:
from statsmodels.tsa.stattools import adfuller, kpss
def kpss_test(series, **kw):    
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')
    return critical_values

critical_values=kpss_test(df['IPG2211A2N'])

KPSS Statistic: 2.206192329752073
p-value: 0.01
num lags: 17
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary


/home/rupeshr/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
/home/rupeshr/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1907: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warn_msg.format(direction="smaller"), InterpolationWarning


In [17]:
for x in critical_values:
    critical_values[x] = round(critical_values[x], 3)

In [21]:
critical_values

{'10%': 0.347, '5%': 0.463, '2.5%': 0.574, '10_pct': 0.739}

In [33]:
critical_values

{'10%': 0.347, '5%': 0.463, '2.5%': 0.574, '1%': 0.739}

In [ ]:
critical_values['10_pct'] = critical_values.pop("1%")

In [34]:

for x in critical_values:
    critical_values[x] = round(critical_values[x], 3)

critical_values['1_pct'] = critical_values.pop("1%")
#critical_values['2.5_pct'] = critical_values.pop("2.5%")
critical_values['5_pct'] = critical_values.pop("5%")
critical_values['10_pct'] = critical_values.pop("10%")

In [37]:
critical_values.pop("2.5%")

0.574

In [39]:
critical_values

{'1_pct': 0.739, '5_pct': 0.463, '10_pct': 0.347}

In [46]:
import matplotlib.dates as mdates
plt.style.use('Solarize_Light2')
from statsmodels.tsa.vector_ar.vecm import coint_johansen
def check_stationary(df,targetparameter,check_stationary_type, differencing):
    if check_stationary_type == 'Rolling_statistics':
        if differencing == 'no':
            w = int(input('Enter Windowing Number:'))
            df['rolling_mean'] = df[targetparameter].rolling(window=w).mean( )
            df['rolling_std'] = df[targetparameter].rolling(window=w).std( )
            return df.index ,df
        elif differencing == 'yes':
            w = int(input(' Enter Windowing Number:'))
            s = int(input(' No.Of.Differencing Required:'))
            df1 = df - df.shift(s)
            df1['rolling_mean'] = df1[targetparameter].rolling(window=w).mean( )
            df1['rolling_std'] = df1[targetparameter].rolling(window=w).std( )
            return df1.index, df1

    elif check_stationary_type == 'ADFTest':
        if differencing == 'no':
            dftest = adfuller(df[targetparameter], autolag='AIC')
            print(dftest)
            Test_stats = dftest[0]
            P_value = dftest[1]
            C_value = dftest[4]
            test = pd.DataFrame([Test_stats, P_value, C_value], columns=[check_stationary_type])
            return df,test

        elif differencing == 'yes':
            s = int(input('No.Of.Differencing Required:'))
            df1 = df[targetparameter] - df[targetparameter].shift(s)
            dftest = adfuller(df1.dropna( ), autolag='AIC')
            Test_stats = dftest[0]
            P_value = dftest[1]
            C_value = dftest[4]
            test = pd.DataFrame([Test_stats, P_value, C_value], columns=[check_stationary_type])
            return df1,test
    
    elif check_stationary_type == 'KPSS': #for this test P value > 0.05 is significant ...just opposite of ADFTest
        if differencing == 'no':
            dftest = kpss(df[targetparameter])
            print(dftest)
            Test_stats = dftest[0]
            P_value = dftest[1]
            C_value = dftest[3]
            test = pd.DataFrame([Test_stats, P_value, C_value], columns=[check_stationary_type])
            return df,test
        
        elif differencing == 'yes':
            s = int(input('No.Of.Differencing Required:'))
            df1 = df[targetparameter] - df[targetparameter].shift(s)
            dftest = kpss(df1.dropna())
            Test_stats = dftest[0]
            P_value = dftest[1]
            C_value = dftest[3]
            test = pd.DataFrame([Test_stats, P_value, C_value], columns=[check_stationary_type])
            return df1,test
                
    elif check_stationary_type =='JohansenTest':
        if differencing =='no':
            det_order=int(input('Enter Det_order:')) # must be in -1 to 1 --> (-1,0,1)
            #k_ar_diff =int(input('Enter Number of lagged differences in the model:')) # --> lag values
            data=df[targetparameter]
            joh_model = coint_johansen(data,det_order,0)
            trace_stats = pd.DataFrame([joh_model.lr1],index=["trace_stat"])
            trace_stats=trace_stats.T
            cvt=pd.DataFrame(joh_model.cvt,columns=['conf_interval_90%','conf_interval_95%','conf_interval_99%'])
            return trace_stats,cvt
        elif differencing =='yes':
            det_order=int(input('Enter Det_order:')) # must be in -1 to 1 --> (-1,0,1)
            k_ar_diff =int(input('Enter Number of lagged differences in the model:')) # --> lag values
            data=df[targetparameter]
            joh_model = coint_johansen(data,det_order,k_ar_diff)
            trace_stats = pd.DataFrame([joh_model.lr1],index=["trace_stat"])
            trace_stats=trace_stats.T
            cvt=pd.DataFrame(joh_model.cvt,columns=['conf_interval_90%','conf_interval_95%','conf_interval_99%'])
            return trace_stats,cvt
targetparameter='IPG2211A2N'
check_stationary_type='KPSS' # Rolling_statistics(use windowing) or ADFTest
differencing='yes' # yes or no
df1,output=check_stationary(df,targetparameter,check_stationary_type,differencing)

No.Of.Differencing Required:1


/home/rupeshr/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
/home/rupeshr/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:1911: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning


In [47]:
print(df1 , output)

DATE
1985-01-01        NaN
1985-02-01    -1.8332
1985-03-01    -8.2218
1985-04-01    -4.9788
1985-05-01    -2.1563
               ...   
2017-09-01   -10.3158
2017-10-01    -5.0017
2017-11-01     3.7222
2017-12-01    17.3853
2018-01-01    14.6836
Name: IPG2211A2N, Length: 397, dtype: float64                                                 KPSS
0                                           0.145907
1                                                0.1
2  {'10%': 0.347, '5%': 0.463, '2.5%': 0.574, '1%...


In [49]:
round(0.1, 3)

0.1